# trainデータの作成

trainデータは以下のようにする
- 過去5回のレースデータを保持
- 過去のデータが1つも無い場合は削除
- 過去のデータが3回未満である場合は0で穴埋め 

### 過去のデータに，他会場でのレースを使用すべきか悩みどころ...
今回は使用しない

In [1]:
import numpy as np
import pandas as pd
#最大表示列数の指定
pd.set_option('display.max_columns', 300)

In [2]:
race_df = pd.read_csv("./csv/cleaned_race_data.csv", sep=",")

In [3]:
horse_df = pd.read_csv("./csv/cleaned_horse_data.csv", sep=",", low_memory=False)

In [4]:
# race
print(race_df.dtypes)
race_df.tail(3)

race_id                     int64
race_round                  int64
weather                    object
ground_status               int64
date                       object
where_racecourse           object
total_horse_number          int64
frame_number_first          int64
horse_number_first          int64
frame_number_second         int64
horse_number_second         int64
frame_number_third          int64
horse_number_third          int64
tansyo                      int64
hukusyo_first               int64
hukusyo_second              int64
hukusyo_third               int64
wakuren                     int64
umaren                      int64
wide_1_2                    int64
wide_1_3                    int64
wide_2_3                    int64
umatan                      int64
renhuku3                    int64
rentan3                     int64
is_obstacle               float64
ground_type                object
is_left_right_straight     object
distance                    int64
weather_rain  

,race_id,race_round,weather,ground_status,date,where_racecourse,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3,is_obstacle,ground_type,is_left_right_straight,distance,weather_rain,weather_snow
49922,202209030410,10,晴,1,2022-06-26 15:01:00,阪神,16,8,16,5,10,7,14,750,230,140,260,1060,1100,470,1460,560,2650,3230,17270,0.0,ダ,右,1400,0,0
49923,202209030411,11,晴,1,2022-06-26 15:40:00,阪神,18,3,6,5,10,4,7,420,180,250,240,2520,2060,780,710,840,3420,5150,25220,0.0,芝,右,2200,0,0
49924,202209030412,12,晴,1,2022-06-26 16:30:00,阪神,16,3,5,7,14,8,15,990,290,1520,660,9570,34170,6890,3370,10540,52400,137920,1016060,0.0,ダ,右,1200,0,0


In [5]:
# horse
horse_df['date'] = pd.to_datetime(horse_df['date'])
print(horse_df.dtypes)
horse_df.head(3)

race_id                        int64
rank                           int64
frame_number                   int64
horse_number                   int64
horse_id                       int64
age                            int64
burden_weight                float64
rider_id                       int64
goal_time                    float64
half_way_rank                float64
last_time                    float64
odds                         float64
popular                      float64
horse_weight                 float64
tamer_id                       int64
owner_id                      object
date                  datetime64[ns]
is_down                        int64
is_senba                       int64
is_mesu                        int64
is_osu                         int64
distance                       int64
avg_velocity                 float64
horse_weight_dif             float64
burden_weight_rate           float64
dtype: object


,race_id,rank,frame_number,horse_number,horse_id,age,burden_weight,rider_id,goal_time,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate
0,200806010101,1,1,2,2005102028,3,54.0,733,73.9,1.0,39.2,8.1,4.0,450.0,138,498800,2008-01-05 09:50:00,0,0,1,0,1200,16.238160,-10.0,0.120000
1,200806010101,2,3,5,2005101044,3,54.0,663,73.9,8.5,38.5,13.4,6.0,482.0,1088,163002,2008-01-05 09:50:00,0,0,1,0,1200,16.238160,2.0,0.112033
2,200806010101,3,2,3,2005104156,3,54.0,684,74.2,10.5,38.5,8.2,5.0,458.0,1054,486800,2008-01-05 09:50:00,0,0,1,0,1200,16.172507,4.0,0.117904


In [6]:
# 全レースコース
pd.unique(race_df['where_racecourse'])

array(['中山', '京都', '小倉', '東京', '中京', '阪神', '福島', '新潟', '函館', '札幌'],
      dtype=object)

## raceからの追加

In [7]:
# raceからのmerge
# 'distance'はすでにある
# 'ground_type','is_left_right_straight' は使用しない
race_df_for_merge = race_df[['race_id',"weather",'ground_status','where_racecourse','total_horse_number','is_obstacle','weather_rain','weather_snow']]

merged_horse_df = pd.merge(horse_df, race_df_for_merge, on='race_id')
merged_horse_df = pd.get_dummies(merged_horse_df, columns=["weather"])

In [8]:
merged_horse_df.head(2)

,race_id,rank,frame_number,horse_number,horse_id,age,burden_weight,rider_id,goal_time,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate,ground_status,where_racecourse,total_horse_number,is_obstacle,weather_rain,weather_snow,weather_小雨,weather_小雪,weather_晴,weather_曇,weather_雨,weather_雪
0,200806010101,1,1,2,2005102028,3,54.0,733,73.9,1.0,39.2,8.1,4.0,450.0,138,498800,2008-01-05 09:50:00,0,0,1,0,1200,16.23816,-10.0,0.120000,1,中山,16,0.0,0,0,0,0,0,1,0,0
1,200806010101,2,3,5,2005101044,3,54.0,663,73.9,8.5,38.5,13.4,6.0,482.0,1088,163002,2008-01-05 09:50:00,0,0,1,0,1200,16.23816,2.0,0.112033,1,中山,16,0.0,0,0,0,0,0,1,0,0


## 標準化・正規化しやすいように先にfloatへ

In [9]:
merged_horse_df[merged_horse_df.columns[(merged_horse_df.columns != 'race_id')&(merged_horse_df.columns != 'horse_id')&(merged_horse_df.columns != 'date')&(merged_horse_df.columns != 'owner_id')&(merged_horse_df.columns != 'tamer_id')&(merged_horse_df.columns != 'rider_id')&(merged_horse_df.columns != 'where_racecourse')]] = merged_horse_df[merged_horse_df.columns[(merged_horse_df.columns != 'race_id')&(merged_horse_df.columns != 'horse_id')&(merged_horse_df.columns != 'date')&(merged_horse_df.columns != 'owner_id')&(merged_horse_df.columns != 'tamer_id')&(merged_horse_df.columns != 'rider_id')&(merged_horse_df.columns != 'where_racecourse')]].astype(float)

In [10]:
print(merged_horse_df.dtypes)
merged_horse_df.shape

race_id                        int64
rank                         float64
frame_number                 float64
horse_number                 float64
horse_id                       int64
age                          float64
burden_weight                float64
rider_id                       int64
goal_time                    float64
half_way_rank                float64
last_time                    float64
odds                         float64
popular                      float64
horse_weight                 float64
tamer_id                       int64
owner_id                      object
date                  datetime64[ns]
is_down                      float64
is_senba                     float64
is_mesu                      float64
is_osu                       float64
distance                     float64
avg_velocity                 float64
horse_weight_dif             float64
burden_weight_rate           float64
ground_status                float64
where_racecourse              object
t

(711250, 37)

## 季節情報としてsin,cosを入れる
月と日のデータを使用して季節のsin,cos

hourでその日の時刻も追加（単位はhour）
(minute, second は,実際の予測データに含めることが困難なので，学習データには含めない．)

In [11]:
from tqdm.notebook import tqdm

merged_horse_df = merged_horse_df.assign(season_cos = 0.0)
merged_horse_df = merged_horse_df.assign(season_sin = 0.0)
merged_horse_df = merged_horse_df.assign(time_in_the_day = 0)
for index, one_horse_data in tqdm(merged_horse_df.iterrows()):
    date = one_horse_data['date']
    season_float = (int(date.month) + (int(date.day) / 31) - 1 ) / 12
    merged_horse_df["season_cos"][index] = np.cos(2 * np.pi * season_float)
    merged_horse_df["season_sin"][index] = np.sin(2 * np.pi * season_float)
    time_in_the_day = int(date.hour)
    merged_horse_df["time_in_the_day"][index] = time_in_the_day
    

0it [00:00, ?it/s]

c:\Users\koich\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":
c:\Users\koich\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
c:\Users\koich\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

# ラベルエンコーディング

In [12]:
from sklearn.preprocessing import LabelEncoder

merged_horse_df['tamer_id'] = LabelEncoder().fit_transform(merged_horse_df['tamer_id'])
merged_horse_df['owner_id'] = LabelEncoder().fit_transform(merged_horse_df['owner_id'])
merged_horse_df['rider_id'] = LabelEncoder().fit_transform(merged_horse_df['rider_id'])

merged_horse_df["tamer_id"] = merged_horse_df["tamer_id"].astype("category")
merged_horse_df["owner_id"] = merged_horse_df["owner_id"].astype("category")
merged_horse_df["rider_id"] = merged_horse_df["rider_id"].astype("category")


# データをレースごとに分ける

In [13]:
# 全レースコース
unique_racename = pd.unique(merged_horse_df['where_racecourse'])
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

for unique, alphabet_name in zip(unique_racename, racecource_name):
    exec("merged_horse_df_{} = merged_horse_df[merged_horse_df['where_racecourse'] == unique].drop(['where_racecourse'], axis=1)".format(alphabet_name))


## 馬の前回までのレース情報をshiftして結合
追加項目
- 前回のレース日時からの差
- rider_idの変化があるか(あるなら1)

In [14]:
# group by したデータフレームに対して行う処理
def make_one_horse_train_data(one_horse_data):
    one_horse_data = one_horse_data.sort_values('date',ascending=False)
    one_horse_data['pre_date_diff'] = one_horse_data['date'].diff(-1).dt.days
    one_horse_data['is_rider_same'] = (one_horse_data['rider_id'].shift(-1) == one_horse_data['rider_id']) * 1.0
    one_horse_data['is_tamer_same'] = (one_horse_data['tamer_id'].shift(-1) == one_horse_data['tamer_id']) * 1.0
    one_horse_data['is_owner_same'] = (one_horse_data['owner_id'].shift(-1) == one_horse_data['owner_id']) * 1.0


    #  数レース分の情報を結合
    result_df = one_horse_data.copy()
    
    for i in range(1,4):
        # i だけ shiftしたものを得る
        # 過去データとしては必要ないものは削除
        shift_df = one_horse_data.drop(['race_id','is_senba','is_mesu','is_osu', 'tamer_id', 'owner_id', 'horse_id'], axis=1).shift(-i)
        # 目的のレースとどれだけの時間離れているか？
        shift_df['interval_date'] = (one_horse_data['date'] - shift_df['date']).dt.days
        shift_df.drop(['date'], axis=1, inplace=True)
        shift_df.columns = shift_df.columns + "_" + str(i)
        result_df = pd.concat([result_df, shift_df], axis=1)
    return result_df


In [17]:
from tqdm import tqdm

def merge_previous_game(merged_horse_df):
    final_df = merged_horse_df.sort_values('date',ascending=False).groupby('horse_id', as_index=False).apply(make_one_horse_train_data)
    return final_df

racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

for alphabet_name in tqdm(racecource_name):
    exec("final_df_{} = merged_horse_df_{}.sort_values('date',ascending=False).groupby('horse_id', as_index=False).apply(make_one_horse_train_data)".format(alphabet_name, alphabet_name))


In [18]:
final_df_nakayama.head()

,race_id,rank,frame_number,horse_number,horse_id,age,burden_weight,rider_id,goal_time,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,is_obstacle,weather_rain,weather_snow,weather_小雨,weather_小雪,weather_晴,weather_曇,weather_雨,weather_雪,season_cos,season_sin,time_in_the_day,pre_date_diff,is_rider_same,is_tamer_same,is_owner_same,rank_1,frame_number_1,horse_number_1,age_1,burden_weight_1,rider_id_1,goal_time_1,half_way_rank_1,last_time_1,odds_1,popular_1,horse_weight_1,is_down_1,distance_1,avg_velocity_1,horse_weight_dif_1,burden_weight_rate_1,ground_status_1,total_horse_number_1,is_obstacle_1,weather_rain_1,weather_snow_1,weather_小雨_1,weather_小雪_1,weather_晴_1,weather_曇_1,weather_雨_1,weather_雪_1,season_cos_1,season_sin_1,time_in_the_day_1,pre_date_diff_1,is_rider_same_1,is_tamer_same_1,is_owner_same_1,interval_date_1,rank_2,frame_number_2,horse_number_2,age_2,burden_weight_2,rider_id_2,goal_time_2,half_way_rank_2,last_time_2,odds_2,popular_2,horse_weight_2,is_down_2,distance_2,avg_velocity_2,horse_weight_dif_2,burden_weight_rate_2,ground_status_2,total_horse_number_2,is_obstacle_2,weather_rain_2,weather_snow_2,weather_小雨_2,weather_小雪_2,weather_晴_2,weather_曇_2,weather_雨_2,weather_雪_2,season_cos_2,season_sin_2,time_in_the_day_2,pre_date_diff_2,is_rider_same_2,is_tamer_same_2,is_owner_same_2,interval_date_2,rank_3,frame_number_3,horse_number_3,age_3,burden_weight_3,rider_id_3,goal_time_3,half_way_rank_3,last_time_3,odds_3,popular_3,horse_weight_3,is_down_3,distance_3,avg_velocity_3,horse_weight_dif_3,burden_weight_rate_3,ground_status_3,total_horse_number_3,is_obstacle_3,weather_rain_3,weather_snow_3,weather_小雨_3,weather_小雪_3,weather_晴_3,weather_曇_3,weather_雨_3,weather_雪_3,season_cos_3,season_sin_3,time_in_the_day_3,pre_date_diff_3,is_rider_same_3,is_tamer_same_3,is_owner_same_3,interval_date_3
701862,202206030812,13.0,7.0,11.0,2016102362,6.0,53.0,30,155.7,1.25,38.4,155.5,12.0,486.0,240,2344,2022-04-17 16:25:00,0.0,0.0,0.0,1.0,2500.0,16.056519,4.0,0.109053,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.283206,0.959059,16,42.0,1.0,1.0,1.0,15.0,6.0,12.0,6.0,57.0,30,134.6,1.25,39.2,284.9,15.0,482.0,0.0,2200.0,16.344725,-2.0,0.118257,1.0,16.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.409820,0.912166,14.0,76.0,1.0,1.0,1.0,42.0,13.0,7.0,11.0,5.0,57.0,30,114.5,7.75,40.2,135.2,14.0,484.0,0.0,1800.0,15.720524,4.0,0.117769,4.0,14.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.979530,-0.201299,14.0,258.0,1.0,1.0,1.0,119.0,13.0,6.0,9.0,5.0,57.0,30,121.9,13.5,34.7,81.6,12.0,474.0,0.0,2000.0,16.406891,-10.0,0.120253,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.067510,0.997719,15.0,70.0,1.0,1.0,1.0,378.0
701856,202206030812,7.0,7.0,10.0,2018103563,4.0,55.0,243,154.3,8.00,35.4,4.2,2.0,492.0,281,3140,2022-04-17 16:25:00,0.0,0.0,0.0,1.0,2500.0,16.202203,-6.0,0.111789,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.283206,0.959059,16,85.0,0.0,1.0,1.0,3.0,3.0,3.0,4.0,54.0,114,121.6,8.50,34.8,10.2,5.0,498.0,0.0,2000.0,16.447368,2.0,0.108434,1.0,11.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.931753,0.363094,15.0,28.0,0.0,1.0,1.0,85.0,6.0,4.0,4.0,3.0,53.0,243,154.5,5.75,35.5,3.3,2.0,496.0,0.0,2500.0,16.181230,4.0,0.106855,4.0,10.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.994869,-0.101168,15.0,91.0,0.0,1.0,1.0,113.0,1.0,6.0,8.0,3.0,54.0,156,151.9,4.0,35.5,8.6,4.0,482.0,0.0,2500.0,16.458196,-8.0,0.112033,1.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.101168,-0.994869,15.0,187.0,0.0,1.0,1.0,204.0
701850,202206030812,1.0,1.0,1.0,2016104364,6.0,57.0,333,152.8,2.75,34.5,3.0,1.0,494.0,266,1935,2022-04-17 16:25:00,0.0,0.0,0.0,1.0,2500.0,16.361257,-4.0,0.115385,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.283206,0.959059,16,85.0,1.0,1.0,1.0,2.0,1.0,1.0,6.0,56.0,333,121.4,10.25,34.4,2.2,1.0,498.0,0.0,2000.0,16.474465,-2.0,0.112450,1.0,11.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.931753,0.363094,15.0,124.0,0.0,0.0,1.0,85.0,1.0,6.0,6.

In [19]:
# 正解ラベル
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

for alphabet_name in racecource_name:
    exec("final_df_{}['is_tansyo'] = (final_df_{}['rank']<=1.0) * 1".format(alphabet_name, alphabet_name))
    exec("final_df_{}['is_hukusyo'] = (final_df_{}['rank']<=3.0) * 1".format(alphabet_name, alphabet_name))


In [20]:
"""
# もともとのdate情報, race_idはあとで必要なので保持
# 予想したいときのレース情報は
- frame_number
- horse_number
- age
- sex['is_senba','is_mesu','is_osu']
- burden_weight
- horse_weight
- distance
- horse_weight_dif
- ground_status
- total_horse_number
- is_obstacle
- 天気やコースについて
- pre_date_diff
- is_rider_same_1	is_tamer_same_1	is_owner_same_1	interval_date_1

# あるとまずい情報は
- rank
- goal_time
- last_time
- odds
- popular
- is_down
- avg_velocity
- half_way_rank
- is_down

rank は正解ラベルを付けたあとに落とす

"""

racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

for alphabet_name in racecource_name:
    exec("final_df_{} = final_df_{}.reset_index(drop=True).sort_values('date')".format(alphabet_name, alphabet_name))
    exec("final_df_{} = final_df_{}.drop(['rank', 'goal_time', 'half_way_rank', 'is_down','last_time', 'odds', 'popular', 'avg_velocity'], axis=1)".format(alphabet_name, alphabet_name))


In [21]:
# 1つ前から先のレースデータがnullの馬がいるようなレースだったら削除する
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

for alphabet_name in racecource_name:
    exec("print('null:' + str(final_df_{}['rank_1'].isnull().sum()) + ' shape:' + str(final_df_{}.shape))".format(alphabet_name, alphabet_name))
    exec("invalid_race = final_df_{}.loc[final_df_{}['rank_1'].isnull(), 'race_id'].unique()".format(alphabet_name, alphabet_name))
    print(invalid_race)
    exec("deleted_df_{} = final_df_{}[~final_df_{}['race_id'].isin(invalid_race)].copy()".format(alphabet_name, alphabet_name, alphabet_name))
    exec("print('null:' + str(deleted_df_{}['rank_1'].isnull().sum()) + ' shape:' + str(deleted_df_{}.shape))".format(alphabet_name, alphabet_name))


null:35872 shape:(102632, 145)
[200806010101 200806010102 200806010103 ... 202206030810 202206030811
 202206030812]
null:0 shape:(9003, 145)
null:29261 shape:(95698, 145)
[200808010101 200808010102 200808010103 ... 202008040806 202008040809
 202008040812]
null:0 shape:(16326, 145)
null:25694 shape:(59458, 145)
[200810010101 200810010102 200810010103 ... 202210020610 202210020611
 202210020612]
null:0 shape:(2190, 145)
null:36446 shape:(110809, 145)
[200805010101 200805010102 200805010103 ... 202205030810 202205030811
 202205030812]
null:0 shape:(12686, 145)
null:30556 shape:(64170, 145)
[200807010101 200807010102 200807010103 ... 202207040410 202207040411
 202207040412]
null:0 shape:(1500, 145)
null:33819 shape:(103309, 145)
[200809010101 200809010102 200809010103 ... 202209030408 202209030409
 202209030412]
null:0 shape:(14255, 145)
null:26486 shape:(48600, 145)
[200803010101 200803010102 200803010103 ... 202203010610 202203010611
 202203010612]
null:0 shape:(431, 145)
null:33604 shap

# 血統データの追加

In [22]:
peds_df = pd.read_csv('csv/pedigree.csv', sep=',', low_memory=False)

In [24]:
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

for alphabet_name in racecource_name:
    exec("deleted_df_{} = pd.merge(deleted_df_{}, peds_df, on='horse_id')".format(alphabet_name, alphabet_name))

In [25]:
deleted_df_tokyo.head(3)

,race_id,frame_number,horse_number,horse_id,age,burden_weight,rider_id,horse_weight,tamer_id,owner_id,date,is_senba,is_mesu,is_osu,distance,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,is_obstacle,weather_rain,weather_snow,weather_小雨,weather_小雪,weather_晴,weather_曇,weather_雨,weather_雪,season_cos,season_sin,time_in_the_day,pre_date_diff,is_rider_same,is_tamer_same,is_owner_same,rank_1,frame_number_1,horse_number_1,age_1,burden_weight_1,rider_id_1,goal_time_1,half_way_rank_1,last_time_1,odds_1,popular_1,horse_weight_1,is_down_1,distance_1,avg_velocity_1,horse_weight_dif_1,burden_weight_rate_1,ground_status_1,total_horse_number_1,is_obstacle_1,weather_rain_1,weather_snow_1,weather_小雨_1,weather_小雪_1,weather_晴_1,weather_曇_1,weather_雨_1,weather_雪_1,season_cos_1,season_sin_1,time_in_the_day_1,pre_date_diff_1,is_rider_same_1,is_tamer_same_1,is_owner_same_1,interval_date_1,rank_2,frame_number_2,horse_number_2,age_2,burden_weight_2,rider_id_2,goal_time_2,half_way_rank_2,last_time_2,odds_2,popular_2,horse_weight_2,is_down_2,distance_2,avg_velocity_2,horse_weight_dif_2,burden_weight_rate_2,ground_status_2,total_horse_number_2,is_obstacle_2,weather_rain_2,weather_snow_2,weather_小雨_2,weather_小雪_2,weather_晴_2,weather_曇_2,weather_雨_2,weather_雪_2,season_cos_2,season_sin_2,time_in_the_day_2,pre_date_diff_2,is_rider_same_2,is_tamer_same_2,is_owner_same_2,interval_date_2,rank_3,frame_number_3,horse_number_3,age_3,burden_weight_3,rider_id_3,goal_time_3,half_way_rank_3,last_time_3,odds_3,popular_3,horse_weight_3,is_down_3,distance_3,avg_velocity_3,horse_weight_dif_3,burden_weight_rate_3,ground_status_3,total_horse_number_3,is_obstacle_3,weather_rain_3,weather_snow_3,weather_小雨_3,weather_小雪_3,weather_晴_3,weather_曇_3,weather_雨_3,weather_雪_3,season_cos_3,season_sin_3,time_in_the_day_3,pre_date_diff_3,is_rider_same_3,is_tamer_same_3,is_owner_same_3,interval_date_3,is_tansyo,is_hukusyo,peds_0,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,peds_10,peds_11,peds_12,peds_13,peds_14,peds_15,peds_16,peds_17,peds_18,peds_19,peds_20,peds_21,peds_22,peds_23,peds_24,peds_25,peds_26,peds_27,peds_28,peds_29,peds_30,peds_31,peds_32,peds_33,peds_34,peds_35,peds_36,peds_37,peds_38,peds_39,peds_40,peds_41,peds_42,peds_43,peds_44,peds_45,peds_46,peds_47,peds_48,peds_49,peds_50,peds_51,peds_52,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61
0,200805050308,7.0,10.0,2004110281,4.0,57.0,21,416.0,113,2026,2008-11-15 13:45:00,0.0,0.0,1.0,2100.0,-8.0,0.137019,1.0,12.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.701110,-0.713053,13,265.0,0.0,1.0,1.0,10.0,3.0,6.0,4.0,53.0,172,157.0,12.50,38.6,31.5,8.0,422.0,0.0,2400.0,15.286624,-6.0,0.125592,1.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.612106,0.790776,16.0,NaN,0.0,0.0,0.0,265.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,モンジュー Montjeu(愛) 1996 鹿毛 [血統][産駒] Sadler's Wells系,Dunshara 1998 鹿毛 [血統][産駒] FNo.[9-c],Sadler's Wells (米) 1981 鹿毛 [血統][産駒],Floripedes 1985 鹿毛 [血統][産駒],Dayjur 1987 黒鹿毛 [血統][産駒],Magnificent Star 1988 鹿毛 [血統][産駒],Northern Dancer 1961 鹿毛 [血統][産駒],Fairy Bridge 1975 鹿毛 [血統][産駒],Top Ville 1976 鹿毛 [血統][産駒],Toute Cy 1979 鹿毛 [血統][産駒],Danzig 1977 鹿毛 [血統][産駒],Gold Beauty 1979 鹿毛 [血統][産駒],Silver Hawk 1979 鹿毛 [血統][産駒],Gulanar 1974 鹿毛 [血統][産駒],Nearctic 1954 黒鹿毛,Natalma 1957 鹿毛,Bold Reason 1968 鹿毛,Special 1969 鹿毛,High Top 1969 鹿毛,Sega Ville 1968 鹿毛,Tennyson 1970 鹿毛,Adele Toumignon 1971 芦毛,Northern Dancer 1961 鹿毛,Pas de Nom 1968 黒鹿毛,Mr. Prospector 1970 鹿毛,Stick to Beauty 1973 黒鹿毛,Roberto 1969 鹿毛,Gris Vitesse 1966 芦毛,Val de Loir 1959 鹿毛,Gulab 1966 鹿毛,Nearco,Lady Angela,Native Dancer,Almahmoud,Hail to Reason,Lalun,Forli,Thong,Derring-Do,Camenae,Charlottesville,La Sega,Val de Loir,Tidra,ゼダーン,Alvorada,Nearctic,Natalma,Admiral's Voya

In [28]:
# 血統データのラベルエンコーディング
from sklearn.preprocessing import LabelEncoder

racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

for alphabet_name in racecource_name:
    exec("deleted_df_{}['horse_id'] = LabelEncoder().fit_transform(deleted_df_{}['horse_id'])".format(alphabet_name, alphabet_name))
    exec("deleted_df_{}['horse_id'] = deleted_df_{}['horse_id'].astype('category')".format(alphabet_name, alphabet_name))
    for i in range(62):
        col_name = 'peds_{}'.format(i)
        exec("deleted_df_{}[col_name] = LabelEncoder().fit_transform(deleted_df_{}[col_name])".format(alphabet_name, alphabet_name))
        exec("deleted_df_{}[col_name] = LabelEncoder().fit_transform(deleted_df_{}[col_name].astype('category'))".format(alphabet_name, alphabet_name))
        

In [29]:
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']

# 0で欠損値を穴埋め
# for alphabet_name in racecource_name:
#     exec("deleted_df_{}.fillna(0, inplace=True)".format(alphabet_name))


# # 欠損値があるデータは削除
# for alphabet_name in racecource_name:
#     exec("print('欠損値削除前のデータ件数: {}'.format(deleted_df_{}.size))".format(alphabet_name))
#     exec("deleted_df_{} = deleted_df_{}.dropna()".format(alphabet_name))
#     exec("print('欠損値削除後のデータ件数: {}'.format(deleted_df_{}.size))".format(alphabet_name))


In [30]:
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']
for alphabet_name in racecource_name:
    exec("deleted_df_{}.isnull().sum()".format(alphabet_name))

In [31]:
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']
for alphabet_name in racecource_name:
    exec("deleted_df_{}.sort_values('date').head()".format(alphabet_name))
    

In [32]:
# データの保存
racecource_name = ['nakayama', 'kyoto', 'ogura', 'tokyo', 'tyukyo', 'hanshin', 'hukushima', 'niigata', 'hakodate', 'sapporo']
for alphabet_name in racecource_name:
    exec("deleted_df_{}.sort_values('date').to_csv('csv/final_data_{}.csv', index=False)".format(alphabet_name, alphabet_name))


In [1]:
import numpy as np
import pandas as pd
#最大表示列数の指定
pd.set_option('display.max_columns', 300)
final_df = pd.read_csv("csv/final_data_tokyo.csv", low_memory=False)

In [17]:
final_df['is_tansyo'].shape

(12686,)